In [5]:
import pandas as pd
import numpy as np
from pprint import pprint

In [6]:
from math import log
import pandas as pd
import random


class CustomDecisionTree():
    def __init__(self):
        pass

    def majorityCnt(self, classList):
        classCount = {}
        for vote in classList:
            if vote not in classCount.keys():
                classCount[vote] = 0
            classCount[vote] += 1
        sortedClassCount = sorted(classCount.items(), reverse=True)
        return sortedClassCount[0][0]

    # for calculting entropy
    def calcShannonEnt(self, dataSet):
        numEntries = len(dataSet)
        labelCounts = {}
        for featVec in dataSet:
            currentLabel = featVec[-1]
            if currentLabel not in labelCounts.keys():
                labelCounts[currentLabel] = 0
                labelCounts[currentLabel] += 1
        shannonEnt = 0.0
        for key in labelCounts:
            prob = float(labelCounts[key])/numEntries
            shannonEnt -= prob * log(prob, 2)
        return shannonEnt

    def splitDataSet(self, dataSet, axis, value):
        retDataSet = []
        for featVec in dataSet:
            if featVec[axis] == value:
                reducedFeatVec = featVec[:axis]
                reducedFeatVec.extend(featVec[axis+1:])
                retDataSet.append(reducedFeatVec)
        return retDataSet

    # choosing the best feature to split
    def chooseBestFeatureToSplit(self, dataSet, labels):
        numFeatures = len(dataSet[0]) - 1
        baseEntropy = self.calcShannonEnt(dataSet)
        bestInfoGain = -1
        bestFeature = 0
        for i in range(numFeatures):
            featList = [example[i] for example in dataSet]
            uniqueVals = set(featList)
            newEntropy = 0.0
            for value in uniqueVals:
                subDataSet = self.splitDataSet(dataSet, i, value)
                prob = len(subDataSet)/float(len(dataSet))
                newEntropy += prob * self.calcShannonEnt(subDataSet)
            infoGain = baseEntropy - newEntropy
            print(infoGain, bestInfoGain)
            if (infoGain > bestInfoGain):
                bestInfoGain = infoGain
                bestFeature = i

        print("the best feature to split is", labels[bestFeature])
        return bestFeature

    # function to build tree recursively
    def createTree(self, dataSet, labels):
        classList = [example[-1] for example in dataSet]
        if len(classList) is 0:
            return

        if classList.count(classList[0]) == len(classList):
            return classList[0]
        if len(dataSet[0]) == 1:
            return self.majorityCnt(classList)
        featureVectorList = [row[:len(row)-1] for row in dataSet]
        bestFeat = self.chooseBestFeatureToSplit(featureVectorList, labels)
        bestFeatLabel = labels[bestFeat]
        myTree = {bestFeatLabel: {}}
        del(labels[bestFeat])
        featValues = [example[bestFeat] for example in dataSet]
        uniqueVals = set(featValues)
        for value in uniqueVals:
            subLabels = labels[:]
            myTree[bestFeatLabel][value] = self.createTree(
                self.splitDataSet(dataSet, bestFeat, value), subLabels)
        return myTree


In [8]:
from scipy.io import arff
import pandas as pd

# Load debug data
lenses = arff.loadarff("lenses.arff")
lenses_df = pd.DataFrame(lenses[0])

str_df = lenses_df.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    lenses_df[col]=str_df[col]
lenses_np = np.array(lenses_df)
lenses_df

FileNotFoundError: [Errno 2] No such file or directory: 'lenses.arff'